In [1]:
import pandas as pd
import sys, os
currentdir = os.path.dirname(os.path.realpath('make_regression_df.py'))
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)
import Delta_Correlation as dc

cptac warning: Your version of cptac (0.9.4) is out-of-date. Latest is 0.9.6. Please run 'pip install --upgrade cptac' to update it. (/Users/humbertogiraldez/Documents/GitHub/pancancerProteinMRNA/notebook_steps_Spearman/Delta_Correlation.py, line 1)


In [98]:
cancers, cancer_names = dc.load_cancers()

cptac warning: The LSCC data is currently strictly reserved for CPTAC investigators. Otherwise, you are not authorized to access these data. Additionally, even after these data become publicly available, they will be subject to a publication embargo (see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details). (/Users/humbertogiraldez/Documents/GitHub/pancancerProteinMRNA/notebook_steps_Spearman/Delta_Correlation.py, line 14)


In [101]:
cancer_dfs = []
for cancer, cancer_name in zip(cancers, cancer_names):    
    if cancer_name == 'CCRCC':
        grade = cancer.get_clinical('tumor')[['histologic_grade']]
        grade['Patient_ID'] = grade.index
        grade = grade.rename(columns = {'histologic_grade': 'Grade'})
        grade = grade.reset_index(level = 0, drop = True)
        grades = []
        for g in grade['Grade']:
            if g == 'G1':
                grades.append('Grade I')
            if g == 'G2':
                grades.append('Grade II')
            if g == 'G3':
                grades.append('Grade III')
            if g == 'G4':
                grades.append('Grade IV')
        grade['Grade'] = grades
    elif cancer_name == 'Endometrial':
        grade = cancer.get_clinical('tumor')[['Histologic_Grade_FIGO']]
        grade['Patient_ID'] = grade.index
        grade = grade.rename(columns = {'Histologic_Grade_FIGO': 'Grade'})
        grade = grade.reset_index(level = 0, drop = True)
        grades = []
        for g in grade['Grade']:
            if g in ['FIGO grade 1']:
                grades.append('Grade I')
            elif g in ['FIGO grade 2']:
                grades.append('Grade II')
            elif g in ['FIGO grade 3']:
                grades.append('Grade III')
            else:
                grades.append(float('NaN'))
        grade['Grade'] = grades
    elif cancer_name == 'HNSCC':
        grade = cancer.get_clinical('tumor')[['histologic_grade']]
        grade['Patient_ID'] = grade.index
        grade = grade.rename(columns = {'histologic_grade': 'Grade'})
        grade = grade.reset_index(level = 0, drop = True)
        grades = []
        for g in grade['Grade']:
            if g in ['G1 Well differentiated']:
                grades.append('Grade I')
            elif g in ['G2 Moderately differentiated']:
                grades.append('Grade II')
            elif g in ['G3 Poorly differentiated']:
                grades.append('Grade III')
            else:
                grades.append(float('NaN'))
        grade['Grade'] = grades
    grade = grade.dropna()
    cancer_df = dc.get_prot_trans_df(cancer)
    cancer_df = cancer_df.dropna()
    cancer_df = pd.merge(cancer_df, grade, how = 'outer')
    cancer_df[['Grade']] = cancer_df[['Grade']].fillna(value='Normal')
    cancer_df = cancer_df.dropna()
    cancer_df['Cancer'] = [cancer_name] * len(cancer_df)
    cancer_dfs.append(cancer_df)
prot_trans_stages_df = pd.concat(cancer_dfs)
prot_trans_stages_df

cptac warning: Due to dropping the specified levels, dataframe now has 907 duplicated column headers. (/Users/humbertogiraldez/Documents/GitHub/pancancerProteinMRNA/notebook_steps_Spearman/Delta_Correlation.py, line 26)
cptac warning: Due to dropping the specified levels, dataframe now has 907 duplicated column headers. (/Users/humbertogiraldez/Documents/GitHub/pancancerProteinMRNA/notebook_steps_Spearman/Delta_Correlation.py, line 36)


,Patient_ID,Gene,Proteomics,Tissue,Transcriptomics,Grade,Cancer
0,C3L-00004,A1BG,-0.304302,Tumor,0.995336,Grade III,CCRCC
1,C3L-00004,A1CF,0.641447,Tumor,16.677828,Grade III,CCRCC
2,C3L-00004,A2M,-0.000025,Tumor,353.263362,Grade III,CCRCC
3,C3L-00004,AAAS,0.207831,Tumor,15.831130,Grade III,CCRCC
4,C3L-00004,AACS,-0.364128,Tumor,2.938550,Grade III,CCRCC
...,...,...,...,...,...,...,...
1454341,C3N-03933.N,ZWILCH,22.136796,Normal,8.900000,Normal,HNSCC
1454342,C3N-03933.N,ZXDC,19.411112,Normal,11.160000,Normal,HNSCC
1454343,C3N-03933.N,ZYG11B,23.439007,Normal,11.960000,Normal,HNSCC
1454344,C3N-03933.N,ZYX,27.619972,Normal,11.330000,Normal,HNSCC


In [102]:
df = prot_trans_stages_df
df = df.drop(columns = 'Patient_ID')
grouped_stages = df.groupby(['Cancer', 'Grade', 'Gene'])
corr_df = grouped_stages.apply(lambda x: x.corr(method = 'spearman', min_periods = 10).iloc[0][1])
corr_df = pd.DataFrame(corr_df, columns = ['Spearman_Rho'])
corr_df.reset_index(inplace = True)
corr_df = corr_df.dropna()
corr_df


,Cancer,Grade,Gene,Spearman_Rho
67,CCRCC,Grade I,ABLIM1,0.442981
69,CCRCC,Grade I,ABLIM3,0.691161
103,CCRCC,Grade I,ACLY,0.310087
115,CCRCC,Grade I,ACOX1,-0.014110
118,CCRCC,Grade I,ACP1,0.623290
...,...,...,...,...
143911,HNSCC,Normal,ZXDC,-0.068118
143912,HNSCC,Normal,ZYG11B,0.809273
143913,HNSCC,Normal,ZYX,0.511977
143914,HNSCC,Normal,ZZEF1,0.129024


In [103]:
corr_df.to_csv('Cancer_grades_correlations.csv', index = False)

In [99]:
cancers = (cancers[0:2] + [cancers[-2]])

In [100]:
cancer_names = cancer_names[0:2] + [cancer_names[-2]]
cancer_names

['CCRCC', 'Endometrial', 'HNSCC']

In [69]:
cancer_dfs = []
for cancer, cancer_name in zip(cancers, cancer_names):    
    if cancer_name == 'CCRCC':
        grade = cancer.get_clinical('tumor')[['histologic_grade']]
        grade['Patient_ID'] = grade.index
        grade = grade.rename(columns = {'histologic_grade': 'Grade'})
        grade = grade.reset_index(level = 0, drop = True)
        grades = []
        for g in grade['Grade']:
            if g == 'G1':
                grades.append('Grade I')
            if g == 'G2':
                grades.append('Grade II')
            if g == 'G3':
                grades.append('Grade III')
            if g == 'G4':
                grades.append('Grade IV')
        grade['Grade'] = grades
    elif cancer_name == 'Endometrial':
        grade = cancer.get_clinical('tumor')[['Histologic_Grade_FIGO']]
        grade['Patient_ID'] = grade.index
        grade = grade.rename(columns = {'Histologic_Grade_FIGO': 'Grade'})
        grade = grade.reset_index(level = 0, drop = True)
        grades = []
        for g in grade['Grade']:
            if g in ['FIGO grade 1']:
                grades.append('Grade I')
            elif g in ['FIGO grade 2']:
                grades.append('Grade II')
            elif g in ['FIGO grade 3']:
                grades.append('Grade III')
            else:
                grades.append(float('NaN'))
        grade['Grade'] = grades
    elif cancer_name == 'HNSCC':
        grade = cancer.get_clinical('tumor')[['histologic_grade']]
        grade['Patient_ID'] = grade.index
        grade = grade.rename(columns = {'histologic_grade': 'Grade'})
        grade = grade.reset_index(level = 0, drop = True)
        grades = []
        for g in grade['Grade']:
            if g in ['G1 Well differentiated']:
                grades.append('Grade I')
            elif g in ['G2 Moderately differentiated']:
                grades.append('Grade II')
            elif g in ['G3 Poorly differentiated']:
                grades.append('Grade III')
            else:
                grades.append(float('NaN'))
        grade['Grade'] = grades
    grade = grade.dropna()
    cancer_df = dc.get_prot_trans_df(cancer)
    cancer_df = cancer_df.dropna()
    cancer_df = pd.merge(cancer_df, grade, how = 'outer')
    cancer_df[['Grade']] = cancer_df[['Grade']].fillna(value='Normal')
    cancer_df = cancer_df.dropna()
    cancer_df['Cancer'] = [cancer_name] * len(cancer_df)
    cancer_dfs.append(cancer_df)

cptac warning: Due to dropping the specified levels, dataframe now has 907 duplicated column headers. (/Users/humbertogiraldez/Documents/GitHub/pancancerProteinMRNA/notebook_steps_Spearman/Delta_Correlation.py, line 26)
cptac warning: Due to dropping the specified levels, dataframe now has 907 duplicated column headers. (/Users/humbertogiraldez/Documents/GitHub/pancancerProteinMRNA/notebook_steps_Spearman/Delta_Correlation.py, line 36)
cptac warning: Due to dropping the specified levels, dataframe now has 745 duplicated column headers. (/Users/humbertogiraldez/Documents/GitHub/pancancerProteinMRNA/notebook_steps_Spearman/Delta_Correlation.py, line 26)
cptac warning: Due to dropping the specified levels, dataframe now has 745 duplicated column headers. (/Users/humbertogiraldez/Documents/GitHub/pancancerProteinMRNA/notebook_steps_Spearman/Delta_Correlation.py, line 36)
cptac warning: Due to dropping the specified levels, dataframe now has 858 duplicated column headers. (/Users/humbertogi